In [11]:
import networkx as nx
import sys
import pandas as pd
import numpy as np
from scipy.special import loggamma
import matplotlib.pyplot as plt

In [21]:
# take the maximum value that a variable can take on
# problem: calculating r values for each variable (take max value of number of choices)

def get_parents(G, node):
    return [neighbor[0] for neighbor in G.in_edges(node)]

def get_children(G, node):
    return [neighbor[1] for neighbor in G.out_edges(node)]

def get_q(G, variables):
    return [np.prod([G.nodes[parent]['r'] for parent in get_parents(G, var)], dtype=int) for var in variables]

def get_r(G, variables):
    return [G.nodes[var]['r'] for var in variables]

def get_j_index(qi, parents_r_values, observed_parents_values, parents):
    one = np.array([1], dtype=int)
    cump = np.cumprod(parents_r_values[:-1])
    k = np.concatenate([one, cump])
    observed_parents_values = np.array(observed_parents_values) - 1
    return (np.dot(k, observed_parents_values)).astype(int)

    observed_parents_values = np.array(observed_parents_values) - 1
    parents_r_values = np.array(parents_r_values)
    indices = np.arange(qi).reshape(parents_r_values)
    # if len(parents_r_values) > 1:
    #     print(indices.shape, observed_parents_values)
    try:
        ind = indices[observed_parents_values].flatten()
        return ind
    except:
        print('ERROR:', parents, indices.shape, observed_parents_values)
        exit()
    # if len(parents_r_values) > 1:
    #     print(ind)
    return ind

def statistics(variables, G, D):
    n = D.shape[1]
    q = get_q(G, variables) 
    M = [np.zeros((q[i], G.nodes[variables[i]]['r']), dtype=int) for i in range(n)]
    # print(q, [m.shape for m in M])
    for obs_i, obs in D.iterrows():
        for i,var in enumerate(variables):
            k = obs[var]-1 # use the observed value of variable as index into M (might not work for all datasets)
            parents = get_parents(G, variables[i])
            j = 0
            if len(parents) > 0:
                j = get_j_index(q[i],
                    [G.nodes[parent]['r'] for parent in parents], 
                    [obs[parent] for parent in parents], parents)
                # j = np.ravel_multi_index(
                #     np.array([G.nodes[parent]['r'] for parent in parents]), 
                #     np.array([obs[parent] for parent in parents]))
            M[i][j,k] += 1
    return M

def prior(variables, G):
    n = len(variables)
    r = get_r(G, variables)
    q = get_q(G, variables) # [np.prod([G.nodes[parent]['r'] for parent in get_parents(G, variables[i])]) for i in range(n)]
    return [np.ones((q[i], r[i])) for i in range(n)] # np.array([np.sum(G[:, i] == '1') for i in range(n)])

def bayesian_score_component(M, alpha):
    p =  np.sum(loggamma(M + alpha)) 
    p -= np.sum(loggamma(alpha)) 
    p += np.sum(loggamma(np.sum(alpha, axis=1)))
    p -= np.sum(loggamma(np.sum(alpha, axis=1) + np.sum(M, axis=1)))
    return p
    # return alpha * np.log(n) - np.sum(np.log(np.arange(1, n + 1)))

def bayesian_score(variables, G, D):
    n = len(variables)
    M = statistics(variables, G, D)
    alpha = prior(variables, G)
    return sum(bayesian_score_component(M[i], alpha[i]) for i in range(n))

def fit(variables, data, filename):
    G = nx.DiGraph()
    for i, v in enumerate(variables):
        G.add_node(v, r=max(data[v]), index=i)
        # G.add_node(v, r=len(data[v].unique()), index=i)

    sorted_variables = list(nx.topological_sort(G))
    for k,i in enumerate(sorted_variables[1:]):
        y = bayesian_score(variables, G, data)
        for _ in range(20):
            y_best, j_best = -np.inf, None
            for j in sorted_variables[:k]:
                if G.has_edge(j, i):
                    continue
                G.add_edge(j, i)
                y_new = bayesian_score(variables, G, data)
                if y_new > y_best:
                    y_best, j_best = y_new, j
                G.remove_edge(j, i)
            if y_best > y:
                print(f'best bayesian score: {y_best} (old: {y})')
                G.add_edge(j_best, i)
                y = y_best
                write_gph(G, {v:v for v in (variables)}, f"data/{filename}.gph")
            else:
                break
    return G

def write_gph(dag, idx2names, filename):
    with open(filename, 'w') as f:
        for edge in dag.edges():
            f.write("{}, {}\n".format(idx2names[edge[0]], idx2names[edge[1]]))

In [22]:

def main():
    # filenames = ['small', 'medium', 'large']
    # for filename in filenames:
    filename = 'small'
    data = pd.read_csv(f"data/{filename}.csv")
    # data = pd.read_csv(f"example/example.csv")
    variables = list(data.columns)
    G = fit(variables, data, filename)
    print('Done training! Final bayesian score:', bayesian_score(variables, G, data))
    # write_gph(G, {v:v for v in (variables)}, f"data/{filename}.gph")
    # write_gph(G, {v:v for v in (variables)}, f"example/example_trained.gph")
    # plot_gph(G, f"data/{filename}.png")

    # nx.write_adjlist(G, "example/example_trained.gph", delimiter=",")
    # print(f'k={k}, y={y}, i={i}, parents={get_parents(G, i)}')

main()
# example for checking bayesian score function
# data = pd.read_csv("example/example.csv")
# variables = list(data.columns)
# edgelist = np.genfromtxt("example/example.gph", delimiter=",", dtype=str)
# G = nx.DiGraph()
# G.add_edges_from(edgelist)
# bayesian_score(variables, G, data)

# G = nx.from_edgelist(edgelist, create_using=nx.DiGraph())
# variables_lookup = {node: i for i, node in enumerate(G.nodes)}
# G['parent1']['r'] = 3 # doesn't work bc it''s a directed graph?


ValueError: invalid entry in coordinates array

In [ ]:
np.array([1,2,3]) - 1

array([0, 1, 2])

In [ ]:
from networkx.drawing.nx_pydot import graphviz_layout

options = {
            "font_size": 10,
            "node_size": 1000,
            "node_color": "white",
            "edgecolors": "gray",
            "linewidths": 5,
            "width": 5,
}
plt.figure()
# pos = nx.shell_layout(G) # use this if graphviz_layout doesn't work
pos = graphviz_layout(G, prog='dot')
nx.draw_networkx(G, pos=pos, arrows=True, arrowstyle='->', **options)

In [ ]:
m = np.zeros((3,4))
m[np.array([0,1]),np.array([2,2])] = 1
m

array([[0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 0.]])